# Импорт библиотек.

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import os
import string
import annoy
import codecs

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm

# Просмотр краткого содержания по данным.

In [272]:
df = pd.read_csv('ProductsDataset.csv')
df.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [274]:
df.isna().sum()

title                0
descrirption      2011
product_id          12
category_id         12
subcategory_id      12
properties          12
image_links         15
dtype: int64

In [275]:
df.fillna(0,inplace=True)

## Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).

In [276]:
df['title'] = [str(value).split(' ') for value in df['title']]
df[['title']]

,title
0,"[Юбка, детская, ORBY]"
1,[Ботильоны]
2,[Брюки]
3,"[Продам, детские, шапки]"
4,[Блузка]
...,...
35543,[Юбка]
35544,"[Новый, твидовый, пиджак]"
35545,"[Женская, зимняя, куртка]"
35546,"[Новая, золотая, ветровка]"


In [277]:
df['descrirption'] = [' '.join(str(value).lower().split(',')) for value in df['descrirption']]
df['descrirption'] = [' '.join(str(value).split('.')) for value in df['descrirption']]
df['descrirption'] = [' '.join(str(value).split('-')) for value in df['descrirption']]
df['descrirption'] = [' '.join(str(value).split('(')) for value in df['descrirption']]
df['descrirption'] = [' '.join(str(value).split(')')) for value in df['descrirption']]
df['descrirption'] = [' '.join(str(value).split('  ')) for value in df['descrirption']]
df['descrirption'] = [str(value).strip(" ") for value in df['descrirption']]
df['descrirption'] = [str(value).split(' ') for value in df['descrirption']]
df['descrirption']

0        [новая, не, носили, ни, разу, в, реале, красив...
1        [новые, привезены, из, чехии, указан, размер, ...
2        [размер, 40, 42, брюки, почти, новые, , не, зн...
3        [продам, шапки, кажда, 200р, розовая, и, белая...
4        [темно, синяя, 42, размер, состояние, отличное...
                               ...                        
35543                [юбка, белая, по, турция, фирма, adl]
35544    [новый, с, бирками, пиджак, размер, s, в, стил...
35545    [женская, зимняя, спортивная, куртка, фирмы, r...
35546    [женская, ветровка, размер, 44, 46, цвет, приг...
35547                                                  [0]
Name: descrirption, Length: 35548, dtype: object

In [278]:
df['properties'] = [' '.join(str(value).split(':')) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("'")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("{")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("}")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("(")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split(")")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split(", ")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("-")) for value in df['properties']]
df['properties'] = ['-'.join(str(value).split("  ")) for value in df['properties']]
df['properties'] = [' '.join(str(value).split("-")) for value in df['properties']]
df['properties'] = [str(value).strip(" ") for value in df['properties']]
df['properties'] = [' '.join(str(value).split("  ")) for value in df['properties']]
df['properties'] = [str(value).split(" ") for value in df['properties']]
df['properties']

0                 [detskie_razmer_rost, 81, 86, 1,5, года]
1        [zhenskaya_odezhda_tzvet, Зеленый, visota_kabl...
2        [zhenskaya_odezhda_dzhinsy_bryuki_tip, Брюки, ...
3        [detskie_pol, Девочкам, detskaya_odezhda_akses...
4        [zhenskaya_odezhda_tzvet, Синий, zhenskaya_ode...
                               ...                        
35543    [zhenskaya_odezhda_platya_yubki_tip, Юбки, zhe...
35544    [brand_zhenskii, Chanel, zhenskaya_odezhda_tzv...
35545    [zhenskaya_odezhda_razmer, 48, 50, XL, zhenska...
35546    [zhenskaya_odezhda_razmer, 44, 46, М, zhenskay...
35547    [zhenskaya_odezhda_aksessuary_tip, Шарфы, и, п...
Name: properties, Length: 35548, dtype: object

In [279]:
df

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,"[Юбка, детская, ORBY]","[новая, не, носили, ни, разу, в, реале, красив...",58e3cfe6132ca50e053f5f82,22.0,2211,"[detskie_razmer_rost, 81, 86, 1,5, года]",http://cache3.youla.io/files/images/360_360/58...
1,[Ботильоны],"[новые, привезены, из, чехии, указан, размер, ...",5667531b2b7f8d127d838c34,9.0,902,"[zhenskaya_odezhda_tzvet, Зеленый, visota_kabl...",http://cache3.youla.io/files/images/360_360/5b...
2,[Брюки],"[размер, 40, 42, брюки, почти, новые, , не, зн...",59534826aaab284cba337e06,9.0,906,"[zhenskaya_odezhda_dzhinsy_bryuki_tip, Брюки, ...",http://cache3.youla.io/files/images/360_360/59...
3,"[Продам, детские, шапки]","[продам, шапки, кажда, 200р, розовая, и, белая...",57de544096ad842e26de8027,22.0,2217,"[detskie_pol, Девочкам, detskaya_odezhda_akses...",http://cache3.youla.io/files/images/360_360/57...
4,[Блузка],"[темно, синяя, 42, размер, состояние, отличное...",5ad4d2626c86cb168d212022,9.0,907,"[zhenskaya_odezhda_tzvet, Синий, zhenskaya_ode...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,[Юбка],"[юбка, белая, по, турция, фирма, adl]",5b5f181c62e1c6616a7f6472,9.0,904,"[zhenskaya_odezhda_platya_yubki_tip, Юбки, zhe...",http://cache3.youla.io/files/images/360_360/5b...
35544,"[Новый, твидовый, пиджак]","[новый, с, бирками, пиджак, размер, s, в, стил...",5bd6c8b29e94ba033d31f8d0,9.0,908,"[brand_zhenskii, Chanel, zhenskaya_odezhda_tzv...",http://cache3.youla.io/files/images/360_360/5b...
35545,"[Женская, зимняя, куртка]","[женская, зимняя, спортивная, куртка, фирмы, r...",5bd6c8bc074b3e1c056f69b2,9.0,903,"[zhenskaya_odezhda_razmer, 48, 50, XL, zhenska...",http://cache3.youla.io/files/images/360_360/5b...
35546,"[Новая, золотая, ветровка]","[женская, ветровка, размер, 44, 46, цвет, приг...",5bd6c8fb2138bbc55745362c,9.0,903,"[zhenskaya_odezhda_razmer, 44, 46, М, zhenskay...",http://cache3.youla.io/files/images/360_360/5b...


In [280]:
for title,descrirption in zip(df[['title']],df[['descrirption']]):
    df[title] = df[title] + df[descrirption]
                     
df[['title']]

,title
0,"[Юбка, детская, ORBY, новая, не, носили, ни, р..."
1,"[Ботильоны, новые, привезены, из, чехии, указа..."
2,"[Брюки, размер, 40, 42, брюки, почти, новые, ,..."
3,"[Продам, детские, шапки, продам, шапки, кажда,..."
4,"[Блузка, темно, синяя, 42, размер, состояние, ..."
...,...
35543,"[Юбка, юбка, белая, по, турция, фирма, adl]"
35544,"[Новый, твидовый, пиджак, новый, с, бирками, п..."
35545,"[Женская, зимняя, куртка, женская, зимняя, спо..."
35546,"[Новая, золотая, ветровка, женская, ветровка, ..."


In [11]:
question = None
written = False
import codecs
from tqdm.notebook import tqdm
#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [12]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [13]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with codecs.open("Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [14]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, size=100, min_count=1, window=5)
model.save("w2v_model")

In [15]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

0it [00:00, ?it/s]

True

In [92]:
def get_answer(chat):
    preprocessed_title = preprocess_txt(chat)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_title:
        if word in model_w2v_chat.wv:
            vector += model_w2v_chat.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index_chat.get_nns_by_vector(vector, 1)
    return index_chat_map[answer_index[0]]

    with open('index_chat_map.pickle', 'rb') as f:
        index_chat_map = pickle.load(f)
    index_chat = annoy.AnnoyIndex(100, 'angular')
    index_chat.load('speaker.ann')